In [ ]:
! pip install psycopg2

In [ ]:
! pip install psycopg2-binary


*https://www.nylas.com/blog/making-use-of-environment-variables-in-python/

* https://stackoverflow.com/questions/40216311/reading-in-environment-variables-from-an-environment-file

In [1]:
import psycopg2

In [2]:
import pandas as pd
import sqlalchemy
import matplotlib as plt
%matplotlib inline

In [ ]:
import os

In [ ]:
home = os.environ['HOME']

In [ ]:
! pip install -U python-dotenv


In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

MY_ENV_VAR = os.getenv('MASTER')

In [ ]:
MY_ENV_VAR

In [ ]:
import os
SECRET_KEY = os.getenv("HELLO")

In [ ]:
print(SECRET_KEY)

In [ ]:
ENV_ADDRESS
ENV_PORT
ENV_USERNAME
ENV_PASSWORD


In [6]:
from sqlalchemy import create_engine
# Postgres username, password, and database name
POSTGRES_ADDRESS = os.getenv("ENV_ADDRESS")  ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = os.getenv("ENV_PORT") 
POSTGRES_USERNAME = os.getenv("ENV_USERNAME")  ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = os.getenv("ENV_PASSWORD") ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = 'Employee_SQL_V2'
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))
# Create the connection
cnx = create_engine(postgres_str)


In [3]:
from sqlalchemy import create_engine
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'localhost' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'postgres' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = 'Class_2'
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))
# Create the connection
cnx = create_engine(postgres_str)


In [8]:
pd.read_sql_query('''SELECT * FROM salaries LIMIT 5;''',cnx)

,emp_id,salary
0,10001,60117
1,10002,65828
2,10003,40006
3,10004,40054
4,10005,78228


https://blog.panoply.io/connecting-jupyter-notebook-with-postgresql-for-python-data-analysis
https://stackoverflow.com/questions/3582552/postgresql-connection-url

In [ ]:
postgresql://localhost:5432